## Radios en Circuitos

**En esta notebook se usan los archivos:**

Radios censales 2010 por IGN

- `radios = './radios_IGN_2010_WGS84/'`

Circuitos compilados por Tartagalensis en base a CNE

- `circuitos_ARG = './mapaelectoral/circuitos-CNE-TTGL.geojson'`

Geometrías de Dptos y Provincias de IGN

- `dptos_url = 'https://dnsg.ign.gob.ar/apps/api/v1/capas-sig/Geodesia+y+demarcación/Límites/departamento/json'`

- `provs_url = 'https://dnsg.ign.gob.ar/apps/api/v1/capas-sig/Geodesia+y+demarcación/Límites/provincia/json'`


**Y se generan los archivos:**

Secciones según circuitos de Tartagalensis - CNE

```python
prov_dpto_IN1[['codprov', 'coddepto', 'IN1']].to_csv('./info/secciones_departamentos-TTGL.csv', index=False)

ref_dptos[['in1prov', 'namprov', 'in1', 'nam', 'IN1', 'geometry']].to_file('./mapaelectoral/ref-dptosIGN.geojson', driver='GeoJSON')

ref_dptos[['in1prov', 'namprov', 'in1', 'nam', 'IN1']].to_csv('./info/ref-dptosIGN.csv', index=False)


In [94]:
import pandas as pd
import geopandas as gpd

import warnings
warnings.filterwarnings('ignore')

from shapely.validation import make_valid

def fix_geom(in_feature):

    # avoid changing original geodf
    in_feature = in_feature.copy(deep=True)    
        
    # drop any missing geometries
    in_feature = in_feature[~(in_feature.is_empty)]
    
    # Repair broken geometries
    for index, row in in_feature.iterrows(): # Looping over all polygons
        if row['geometry'].is_valid:
            next
        else:
            fix = make_valid(row['geometry'])

            try:
                in_feature.loc[[index],'geometry'] =  fix # issue with Poly > Multipolygon
            except ValueError:
                in_feature.loc[[index],'geometry'] =  in_feature.loc[[index], 'geometry'].buffer(0)
    return in_feature

## Cargar geometrias

In [233]:
## Para bajar estos poligonos consultar el repo geoespacial-censo-IGN
# './IGN_shp/ign_departamento'

IGN = gpd.read_file('./../geoespacial-censo-IGN/IGN_shp/ign_departamento')

In [132]:
radios = gpd.read_file('./radios_IGN_2010_WGS84/')
radios.head()


,PROV_,DEPTO_,FRACC_,COD_2010,IN1,geometry
0,02,013,03,020130302,0214091,"POLYGON ((-58.46740 -34.53485, -58.46740 -34.5..."
1,02,013,11,020131101,0214091,"POLYGON ((-58.45868 -34.53715, -58.45868 -34.5..."
2,02,013,01,020130104,0214091,"POLYGON ((-58.46758 -34.53733, -58.46794 -34.5..."
3,02,013,01,020130103,0214091,"POLYGON ((-58.46846 -34.53531, -58.46846 -34.5..."
4,02,013,01,020130102,0214091,"POLYGON ((-58.47269 -34.53719, -58.47269 -34.5..."


In [234]:
# claves

repo = './../geoespacial-censo-IGN'

censo10_IGN_ref = gpd.read_file(repo+'/info/censo10_IGN_ref.geojson') 

In [235]:
censo10_IGN_ref.head()

,PROV_,DEPTO_,IN1,NAM,area_km2,geometry
0,02,001,02007,Comuna 1,0.001742,"POLYGON Z ((-58.39154 -34.58087 0.00000, -58.3..."
1,02,002,02014,Comuna 2,0.000615,"POLYGON Z ((-58.40451 -34.59804 0.00000, -58.4..."
2,02,003,02021,Comuna 3,0.000627,"POLYGON Z ((-58.39991 -34.62857 0.00000, -58.3..."
3,02,004,02028,Comuna 4,0.002131,"MULTIPOLYGON Z (((-58.35579 -34.61814 0.00000,..."
4,02,005,02035,Comuna 5,0.000654,"POLYGON Z ((-58.42926 -34.61512 0.00000, -58.4..."


In [152]:
# circuitos_ARG = gpd.read_file('./mapaelectoral/circuitos-CNE.geojson')
circuitos_ARG = gpd.read_file('./mapaelectoral/circuitos-CNE-TTGL.geojson')
circuitos_ARG = fix_geom(circuitos_ARG.dropna(subset = 'geometry'))
circuitos_ARG.head()

,circuito,codprov,coddepto,seccion,geometry
0,00001,01,001,NaN,"MULTIPOLYGON (((-58.36620 -34.62040, -58.36610..."
1,00002,01,001,NaN,"MULTIPOLYGON (((-58.37740 -34.61740, -58.37740..."
2,00003,01,001,NaN,GEOMETRYCOLLECTION (POLYGON ((-58.35720 -34.59...
3,00004,01,001,NaN,"POLYGON ((-58.36760 -34.60760, -58.36770 -34.6..."
4,00005,01,001,NaN,GEOMETRYCOLLECTION (MULTIPOLYGON (((-58.36080 ...


## Departamentos API Georef

https://datos.gob.ar/ar/dataset/ign-unidades-territoriales/archivo/ign_01.03.02

     OBJECTID  Entidad        Objeto                                   FNA  \
0        4388        0  Departamento  Departamento Islas del Atlántico Sur   
1        4389        0  Departamento                Departamento Concordia   
2        4390        0  Departamento                  Departamento Federal   
3        4391        0  Departamento             Departamento Gualeguaychú   
4        4392        0  Departamento         Departamento Islas del Ibicuy   
..        ...      ...           ...                                   ...   
524      4913        0  Departamento             Departamento Valle Fértil   
525      4914        0  Departamento                   Partido de La Plata   
526      4915        0  Departamento                 Departamento Mercedes   
527      4916        0  Departamento            Departamento Santa Bárbara   
528      4531        0  Departamento                     Departamento Orán   

              GNA                      NAM  \
0    Departamento

In [154]:
import geopandas as gpd
import pandas as pd

def curated_overlay(lower, higher, low_index, high_index):
    # Perform overlay to intersect radios with higher-level polygons
    overlay = gpd.overlay(lower, higher, how='intersection')
    
    result_prov = []  # List to store the results for each higher-level unit
    
    if not overlay.empty:  # If there is an intersection
        # Calculate the area of the intersected polygons
        overlay['area'] = overlay.area

        # # Select the radio with the largest intersection area for each higher-level unit
        # radios_in_circu = ref_dptos.groupby(['PROV_', 'DEPTO_', 'FRACC_', 'COD_2010']).apply(
        #     lambda x: x.nlargest(1, 'area')
        # ).reset_index(drop=True)[['codprov', 'coddepto', 'PROV_', 'DEPTO_', 'FRACC_', 'COD_2010', 'IN1', 'circuito']]

        # Select the radio with the largest intersection area for each higher-level unit
        lower_in_higher = overlay.groupby(low_index).apply(
            lambda x: x.nlargest(1, 'area')
        ).reset_index(drop=True)[low_index + high_index]#[['codprov', 'coddepto', 'seccion', 'IN1', 'NAM', 'circuito', 'COD_2010']]

                # radios_in_circu = ref_dptos.groupby(['PROV_', 'DEPTO_', 'FRACC_', 'COD_'+year]
                #                         ).apply(lambda x: x.nlargest(1, 'area')).reset_index(drop = True
                #     )[['codprov', 'coddepto', 'PROV_', 'DEPTO_', 'FRACC_', 'COD_'+year, 'IN1', 'circuito']]
        


        # result_prov.append(radios_in_circu)
        result_prov.append(lower_in_higher)
    else:
        print('No intersection with any higher-level unit.')

    return result_prov



In [185]:

lower = circuitos_ARG[['circuito', 'codprov', 'coddepto', 'geometry']]
higher = IGN[['IN1', 'NAM', 'geometry']]

result = curated_overlay(lower, higher, low_index = ['codprov', 'coddepto', 'circuito'], high_index = ['IN1', 'NAM'])
print(result)


[     codprov coddepto circuito    IN1         NAM
0         01      001    00001  02007    Comuna 1
1         01      001    00002  02007    Comuna 1
2         01      001    00003  02007    Comuna 1
3         01      001    00004  02007    Comuna 1
4         01      001    00005  02007    Comuna 1
...      ...      ...      ...    ...         ...
5498      24       02     0014  94007  Río Grande
5499      24       02     0016  94007  Río Grande
5500      24       02     0021  94011     Tolhuin
5501      24       02     0210  94007  Río Grande
5502      24       02     0221  94007  Río Grande

[5503 rows x 5 columns]]


In [ ]:

lower = circuitos_ARG[['circuito', 'codprov', 'coddepto', 'geometry']]
higher = IGN[['IN1', 'NAM', 'geometry']]

result = curated_overlay(lower, higher, low_index = ['codprov', 'coddepto', 'circuito'], high_index = ['IN1', 'NAM'])
print(result)


In [223]:
df = result[0]
counts = df.groupby(['codprov', 'coddepto', 'IN1', 'NAM']).size().reset_index(name='counts')

# Get the row indices with the highest counts within each group
idx_max_counts = counts.groupby(['codprov', 'coddepto'])['counts'].idxmax()
# Select the corresponding rows from the original DataFrame using the indices
ref = counts.loc[idx_max_counts]


In [228]:
## Claves 527 deptos, de CNE a IGN
ref.head(5)

,codprov,coddepto,IN1,NAM,counts
0,01,001,02007,Comuna 1,19
1,01,002,02014,Comuna 2,10
2,01,003,02021,Comuna 3,13
3,01,004,02028,Comuna 4,15
4,01,005,02035,Comuna 5,10


In [230]:
# ref.loc[ref.IN1 == '82084']

In [232]:
# codprov	coddepto con mas de un depto IGN:

# Lugares que bordean con otros deptos y pueden dar lugar a error. Circuitos con depto incorrecto.
# duplicates = df.groupby(['codprov', 'coddepto']).agg({'IN1' : ['nunique', 'unique'], 'NAM': 'unique'}).sort_values(('IN1', 'nunique')).tail(20)
# duplicates


In [ ]:
## Asignamos radios y circuitos espacialmente a los departamentos de IGN. Para evitar todo tipo de problemas con los codigos de deptos.

# Radios en Circuitos
#### Interseccion espacial

In [ ]:
circuitos_ARG.count()

circuito    5589
codprov     5563
coddepto    5563
seccion     1066
geometry    5559
dtype: int64

In [ ]:
circuitos_ARG.nunique()

circuito    2058
codprov       24
coddepto     138
seccion        8
geometry    5550
dtype: int64

In [ ]:
## Agregar codigos de provincia de INDEC
codprovs = pd.read_csv('./codprovs.csv', dtype = 'str')
codprovs['codprov'] = codprovs['codprov'].str.zfill(2)

circuitos_ARG = circuitos_ARG.merge(codprovs, how = 'left')

In [ ]:

result_list = []

for indec_p in circuitos_ARG.indec_p.dropna().unique():
    cne_prov = circuitos_ARG.loc[circuitos_ARG.indec_p.fillna(indec_p) == indec_p]
    cne_prov = fix_geom(cne_prov.dropna(subset = 'geometry'))
    print(cne_prov.nomprov.unique())

    radios_prov = radios.loc[radios.PROV_ == indec_p]

#     ## Correccion de errores
#     radios_prov.loc[radios_prov.IN1 == '26028', 'DEPTO_'] = '063' # Esto es para adaptarse a un erorr en CNE
# #             (la CNE tiene unido camarones al dpto martires, por error)

    result_prov = []

#     for dpto in cne_prov.indec_d.unique():

#         if (cne_prov.indec_p.unique()[0] == '94') | (cne_prov.indec_p.unique()[0] == '22'):
# #                     print('chaco o tdf')
#             radios_dpto = radios_prov # En chaco y tdf, usar toda la provincia porque los codigos estan cambiados...
#         else:
#             radios_dpto = radios_prov.loc[radios_prov.DEPTO_ == dpto]

# #         circ_dpto = cne_prov.loc[cne_prov.indec_d == dpto]

# #     Check...
#     fig, axs = plt.subplots(1)
#     radios_prov.plot(ax = axs, alpha = .4)
#     cne_prov.plot(ax = axs, alpha = .4)
#     plt.show()

    ## Intersectar los radios en cuestion con los polis de IGS.
    overlay = gpd.overlay(radios_prov, cne_prov[['circuito', 'codprov', 'coddepto', 'geometry']], how='intersection')
    
    
    if len(overlay) != 0: # Si hay interseccion
        overlay['area'] = overlay.area

        year = '2010'
        # Quedarse con el que mas interseca (el radio se asigna al depto donde este mayor parte de su area.)
        radios_in_circu = overlay.groupby(['PROV_', 'DEPTO_', 'FRACC_', 'COD_'+year]
                                        ).apply(lambda x: x.nlargest(1, 'area')).reset_index(drop = True
                    )[['codprov', 'coddepto', 'PROV_', 'DEPTO_', 'FRACC_', 'COD_'+year, 'IN1', 'circuito']]
        
        # radios_in_circu['FRACC_2'] = radios_in_circu['COD_'+year].str[5:7]
        result_prov += [radios_in_circu]
    else:
#             print('sin interseccion en '+str(circ_dpto.departamen.unique()[0]))
        print('sin interseccion en '+str(cne_prov.nomprov.unique()[0]))

    result_prov = pd.concat(result_prov)

    ## Tomar radios que se asociaron a mas de 1 circuito.
#     Esto pasa cuando se usa la provincia entera para intersecar (ie. chaco, tdf)
    duplicados = result_prov[result_prov.duplicated('COD_2010', keep = False)].sort_values('COD_2010')

    if len(duplicados) > 0:

        ## Volcarlos en 1 solo circuito.
        overlay = gpd.overlay(gpd.GeoDataFrame(duplicados.drop('circuito', axis = 1).merge(radios)), 
                             gpd.GeoDataFrame(duplicados.merge(cne_prov))[['circuito', 'geometry']], how='intersection')

        if len(overlay) != 0: # Si hay interseccion
            overlay['area'] = overlay.area

            year = '2010'
            # Quedarse con el que mas interseca (el radio se asigna al depto donde este mayor parte de su area.)
            res_dup_prov = overlay.groupby(['PROV_', 'DEPTO_', 'FRACC_', 'COD_'+year]
                                        ).apply(lambda x: x.nlargest(1, 'area')
                                               ).reset_index(drop = True)[['PROV_', 'DEPTO_', 'FRACC_', 'COD_'+year, 'IN1', 'circuito']]
        result_prov = pd.concat([result_prov[~result_prov.duplicated('COD_2010', keep = False)], res_dup_prov])

    result_list += [result_prov]

result = pd.concat(result_list).drop_duplicates()


['Ciudad Autónoma de Buenos Aires' nan]
['Catamarca' nan]
['Chaco' nan]
['Chubut' nan]
['Córdoba' nan]
['Corrientes' nan]
[nan 'Entre Ríos']
[nan 'Formosa']
[nan 'Jujuy']
[nan 'La Pampa']
[nan 'La Rioja']
[nan 'Mendoza']
[nan 'Misiones']
[nan 'Neuquén']
[nan 'Buenos Aires']
[nan 'Río Negro']
[nan 'Salta']
[nan 'San Luis']
[nan 'Santa Cruz']
[nan 'Santa Fe']
[nan 'Santiago del Estero']
[nan 'San Juan']
[nan 'Tierra del Fuego, Antártida e Islas del Atlántico Sur']
[nan 'Tucumán']


In [ ]:
radios.nunique()

PROV_          24
DEPTO_        154
FRACC_         99
COD_2010    52401
IN1           525
geometry    52401
dtype: int64

In [ ]:
result.nunique()

codprov        24
coddepto      138
PROV_          24
DEPTO_        154
FRACC_         99
COD_2010    52159
IN1           525
circuito     1974
dtype: int64

In [ ]:
# Esto es lo extraido de la base
# seccion_dpto = pd.read_csv('./../elecciones-ARG/datos/BD/seccion_table.csv', dtype = 'str')
# seccion_dpto['distrito_id'] = seccion_dpto['distrito_id'].str.zfill(2)
# seccion_dpto['seccion_id'] = seccion_dpto['seccion_id'].str.zfill(3)
# seccion_dpto.nunique()

## Este archivo esta basado en lo extraido de la base, pero trae los codigos usados en INDEC, etc (PROV_, DEPTO_)
# result = pd.read_csv('./info/radios_circuitos-TTGL.csv', dtype= 'str')
secciones_dptos_ref = pd.read_csv('./info/secciones_dptos_ref.csv')

In [ ]:
secciones_dptos_ref = secciones_dptos_ref.dropna()
secciones_dptos_ref['PROV_'] = secciones_dptos_ref.IDPROV.astype(int).astype(str).str.zfill(2)
secciones_dptos_ref['DEPTO_'] = secciones_dptos_ref.IDDPTO.astype(int).astype(str).str.zfill(3)

In [ ]:
claves = secciones_dptos_ref[['PROV_', 'DEPTO_', 'distrito_id', 'seccion_id', 'seccionprovincial_id', 'seccion_nombre']]
claves.head()

,PROV_,DEPTO_,distrito_id,seccion_id,seccionprovincial_id,seccion_nombre
0,02,001,1,1,0,Comuna 01
1,02,002,1,2,0,Comuna 02
2,02,003,1,3,0,Comuna 03
3,02,004,1,4,0,Comuna 04
4,02,005,1,5,0,Comuna 05


In [ ]:
result.head()

,codprov,coddepto,PROV_,DEPTO_,FRACC_,COD_2010,IN1,circuito
0,01,001,02,001,01,020010101,0214007,00005
1,01,001,02,001,02,020010201,0214007,00005
2,01,001,02,001,02,020010202,0214007,00005
3,01,001,02,001,02,020010203,0214007,00005
4,01,001,02,001,02,020010204,0214007,00005


In [ ]:
result_c = result.merge(claves)

In [96]:
# result_c.to_csv('./info/radios_circuitos_secciones_ref.csv', index = False)  #### Hay problema con IN1... a revisar. O bien en el codigo que sigue debajo.

In [121]:
result_c.drop('IN1', axis = 1).to_csv('./../elecciones-ARG/datos/censo/radios_circuitos_secciones_ref.csv', index = False)

In [131]:
result[['codprov', 'coddepto', 'PROV_', 'DEPTO_']].drop_duplicates()

,codprov,coddepto,PROV_,DEPTO_
0,01,001,02,001
329,01,002,02,002
527,01,003,02,003
781,01,004,02,004
1033,01,005,02,005
...,...,...,...,...
1474,23,016,90,105
1597,23,014,90,112
1641,23,015,90,119
1647,23,016,90,119


In [125]:
result_c.drop('IN1', axis = 1).head()

,codprov,coddepto,PROV_,DEPTO_,FRACC_,COD_2010,circuito,distrito_id,seccion_id,seccionprovincial_id,seccion_nombre
0,01,001,02,001,01,020010101,00005,1,1,0,Comuna 01
1,01,001,02,001,02,020010201,00005,1,1,0,Comuna 01
2,01,001,02,001,02,020010202,00005,1,1,0,Comuna 01
3,01,001,02,001,02,020010203,00005,1,1,0,Comuna 01
4,01,001,02,001,02,020010204,00005,1,1,0,Comuna 01


In [128]:
result_c[['codprov', 'coddepto', 'PROV_', 'DEPTO_', 'distrito_id', 'seccion_id', 'seccion_nombre']].drop_duplicates()



,codprov,coddepto,PROV_,DEPTO_,distrito_id,seccion_id,seccion_nombre
0,01,001,02,001,1,1,Comuna 01
329,01,002,02,002,1,2,Comuna 02
527,01,003,02,003,1,3,Comuna 03
781,01,004,02,004,1,4,Comuna 04
1033,01,005,02,005,1,5,Comuna 05
...,...,...,...,...,...,...,...
51916,23,016,90,105,23,16,Tafí Viejo
52039,23,014,90,112,23,14,Trancas
52083,23,015,90,119,23,15,Yerba Buena
52089,23,016,90,119,23,15,Yerba Buena


In [ ]:
xx

## Codprov - Coddepto - IN1

In [ ]:
pd.options.display.max_rows = 99

In [ ]:
## Cuando codprov y coddpto tienen mas de un IN1, elegir el que mas radios tiene en la interseccion.

prov_dpto_nuniques = result.groupby(['codprov', 'coddepto', 'IN1']).nunique().reset_index()
prov_dpto_nuniques['pct'] = prov_dpto_nuniques.groupby(['codprov', 'coddepto'])['COD_2010'].apply(lambda x: x/x.sum()).round(2)
prov_dpto_nuniques['test'] = abs(prov_dpto_nuniques['pct'] - .5)

# test.sort_values('test').head(20).sort_values('IN1')

TypeError: incompatible index of inserted column with frame index

In [ ]:
## Lezama y Chascomus
prov_dpto_nuniques.loc[prov_dpto_nuniques.IN1 == '06217']

: 

In [ ]:
## Tierra del Fuego
prov_dpto_nuniques.loc[prov_dpto_nuniques.IN1.str[:2] == '94']

: 

In [ ]:
prov_dpto_IN1 = prov_dpto_nuniques.groupby(['codprov', 'coddepto']).apply(lambda x: x.nlargest(1, 'COD_2010')).reset_index(drop = True)

prov_dpto_IN1[['codprov', 'coddepto', 'IN1']].to_csv('./info/secciones_departamentos-TTGL.csv', index = False)

: 

In [ ]:
prov_dpto_IN1[['codprov', 'coddepto', 'IN1']].head()

: 

## Unir secciones a departamentos IGN

In [ ]:
import geopandas as gpd

# Unir secciones a departamentos IGN
radios_diss_IN1 = radios.copy()
radios_diss_IN1['geometry'] = radios_diss_IN1.buffer(0.001)
radios_diss_IN1 = radios_diss_IN1.dissolve('IN1').reset_index()[['IN1', 'geometry']]

# Si bien en teoria este dissolve nos daria los departmantos, el archivo de radios IGN tiene varios faltantes y errores.
# Por eso es mejor hacer spatial join de los radios disueltos, con la fuente de DPTOS.

: 

### Radios de IGN en DPTOS y PROVS de IGN

In [ ]:
## Si este paso falla, se pueden bajar manualmente los archivos y abrirlos localmente.

try: 
    dptos_url = u'https://dnsg.ign.gob.ar/apps/api/v1/capas-sig/Geodesia+y+demarcación/Límites/departamento/json'
    dptos_IGN = gpd.read_file(dptos_url)

    # Agregar código y nombres de provincia
    provs_url = 'https://dnsg.ign.gob.ar/apps/api/v1/capas-sig/Geodesia+y+demarcación/Límites/provincia/json'
    provs_IGN = gpd.read_file(provs_url)

except:
    dptos_IGN = gpd.read_file('./datos/departamento.json')
    provs_IGN = gpd.read_file('./datos/provincia.json')


: 

In [ ]:

dptos_IN1_from_radios = radios_diss_IN1

overlay = gpd.overlay(dptos_IGN[['nam', 'in1', 'geometry']], dptos_IN1_from_radios, how='intersection')
overlay['area'] = overlay.area

overlay = overlay.groupby(['nam', 'in1']).apply(lambda x: x.nlargest(1, 'area')).reset_index(drop=True)

provnames = provs_IGN.rename(columns={'in1': 'in1prov', 'nam': 'namprov'})[['in1prov', 'namprov']]
overlay['in1prov'] = overlay['in1'].str[:2]
overlay = overlay.merge(provnames, on='in1prov')

overlay.head()

: 

In [ ]:
# Lugares con código diferente (in1, de radios IGN, vs. IN1 de departamentos IGN)
# Son las comunas de CABA, además de Lezama-Chascomús y Tierra del Fuego
overlay.loc[overlay.in1 != overlay.IN1].sort_values('in1')

: 

### Guardar los resultados

In [ ]:
# Geojson
overlay[['in1prov', 'namprov', 'in1', 'nam', 'IN1', 'geometry']].to_file('./mapaelectoral/ref-dptosIGN.geojson', driver='GeoJSON')

# CSV
overlay[['in1prov', 'namprov', 'in1', 'nam', 'IN1']].to_csv('./info/ref-dptosIGN.csv', index=False)

: 

: 